# When Adding Variables to a Regression is Wrong! 

A brief exploration of casual models and why throwing in all the variables you have is a terrible approach.

I'm taking this example from Causal Inference: The Mixtape and using it as a change to practice my Python. I normally code in R, so if anyone has suggestions for improving the Python code, feel free to let me know. 

The example we'll use is gender discrimination in the workforce. So in this case we're trying to estimate the impact of gender discrimination on wages. 

In [1]:
import numpy as np #for generating arrays with random numbers
import pandas as pd #dataframes
import statsmodels.api as sm #to run the actual ols model

np.random.seed(42)


We're going to first generate a labor force that is half female and has ability random distributed. Simulated data is really useful for exploring different model assumptions and specifications because you know what the data looks like. 

In [2]:
generated_data = {
    'female'  : np.random.randint(low = 0, high = 2, size = 10000, dtype = int), #the high argument is not inclusive, so this is randomly generating 0s and 1s. 
    'ability' : np.random.normal(size = 10000),
}

df = pd.DataFrame(data = generated_data)

Now we need to generate some other variables of interest. We're looking at the impact of discrimination, so let's set that to be experienced by the female half of the labor force that we've simulated. We're going to assume that discrimination affects both wages and choice of occupation. Here we're worried about occupations in terms of higher and lower pay scales, so let's set occupations to be positively associated with ability and negatively associated with discrimination. 

Finally, wages are negatively associated with discrimination and positively associated with both occupation and ability. 

In [3]:
df['discrimination'] = df['female']
df['occupation'] = 1 + 2 * df['ability'] + 0 * df['female'] - 2 * df['discrimination'] + np.random.normal(size = 10000)
df['wage'] = 1 - 1 * df['discrimination'] + 1 * df['occupation'] + 2 * df['ability'] + np.random.normal(size = 10000)

df.describe()

,female,ability,discrimination,occupation,wage
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.498700,-0.008041,0.498700,-0.009388,0.471065
std,0.500023,1.004178,0.500023,2.449597,4.545405
min,0.000000,-3.922400,0.000000,-10.018905,-18.328506
25%,0.000000,-0.674327,0.000000,-1.640437,-2.517222
50%,0.000000,-0.007682,0.000000,-0.022777,0.482132
75%,1.000000,0.668901,1.000000,1.633467,3.501387
max,1.000000,3.529055,1.000000,9.500154,16.731628


Now that we have our simulated data with specified causal relationships, let's look at a few different regression models. We'll first look at a model that only includes being female as a cause of wages. 

In [4]:
# Set up matrices for regression
Y = df['wage']
X1 = df['female']
X2 = df[['female', 'occupation']]
X3 = df[['female', 'occupation', 'ability']]

# add constants to each X matrix for the intercept of the model
X1 = sm.add_constant(X1)
X2 = sm.add_constant(X2)
X3 = sm.add_constant(X3)

model1 = sm.OLS(Y, X1)
results1 = model1.fit()
results1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   wage   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     1195.
Date:                Sat, 30 Oct 2021   Prob (F-statistic):          2.09e-247
Time:                        11:50:13   Log-Likelihood:                -28766.
No. Observations:               10000   AIC:                         5.754e+04
Df Residuals:                    9998   BIC:                         5.755e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.9522      0.061     32.173      0.000       1.833       2.071
female        -2.9700      0.086    -34.565      0.000      -3.138      -2.802
==============================================================================
Omnibus:                        2.156   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.340   Jarque-Bera (JB):                2.186
Skew:                          -0.008   Prob(JB):                        0.335
Kurtosis:                       3.071   Cond. No.                         2.62
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

We get a lot here, but what we're mainly interested in is the coefficients, so let's look at those. Here we see that being female has a strong negative impact on wages earned. 

In [5]:
results1.params

const     1.952182
female   -2.969956
dtype: float64

What if we control for occupation? (This is usually suggested when workplaces are facing gender discrimination lawsuits). Remember that we set up occupation to also be impacted by discrimination. So when we add this as a control, the sign actually flips, and all of a sudden it seems like being female leads to higher wages. We know that's not correct in our simulated universe of data - so what happened? The causal model implied in this regression is wrong. It suggests causation runs from occupation to wages and that there's nothing else that also impacts both occupation and wages. However, ability and discrimination also impact occupation and wages, and we're not currently conditioning the model on ability. 

In [6]:
model2 = sm.OLS(Y, X2)
results2 = model2.fit()

model3 = sm.OLS(Y, X3)
results3 = model3.fit()

results2.params

const         0.208846
female        0.559992
occupation    1.815929
dtype: float64

So let's try conditioning on abilty. Here we're back to a clear impact of gender discrimination. 

In [7]:
results3.params

const         0.988717
female       -0.986841
occupation    1.025762
ability       1.975298
dtype: float64

A major problem is that in the real world we can't observe ability directly and put it in a regression model. Another issue is that this causal model is still very incomplete. Nonetheless, the way the sign flips back and forth depending on the model is hopefully an illustration of why it's so important to have a theoretical model and not just throw in as much data as possible. 